# Projet Groover

## Exercie 1

In [1]:
import os 
import re
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.compat.v1.losses import mean_squared_error
from tensorflow.keras.layers import Dense, Dropout, LSTM
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from gensim.models import Word2Vec
from gensim.test.utils import get_tmpfile

Type_influencer=[
        'Media',          
        'Radio',           
        'Label',           
        'Playlist',        
        'Journalist',      
        'Channel',         
        'Booker',          
        'Mentor',          
        'Manager',         
        'Springboard',     
        'Publisher',       
        'Supervisor',
        'Event'  ]

### préparation des données pour l'algorithme Word2Vec

In [3]:
def prepare_DF(DF):
    DF_help=DF.apply(func = lambda S:S.lower())
    DF_help=remplacer_site_DF(DF)
    DF_help=DF_help.apply(func = lambda S:S.strip())
    #DF_help=espacer_point_DF(DF)
    return DF_help

def get_longest_feedback(List):
    return max([len(l) for l in List])

def prepare_sentences_List(DF):
    List=[sentence for paragraph in DF.values.tolist() for sentence in paragraph]
    return List

def prepare_sentences_help(DF):
    DF_help = DF.apply(func = lambda S:[s.split(' ') for s in S.split('.')])
    DF_help = DF_help.apply(func = lambda paragraphe : [ [word for word in sentence if not(word in [' ',''])]for sentence in paragraphe ])
    DF_help = DF_help.apply(func = lambda paragraphe : [ sentence for sentence in paragraphe if not(sentence in [[' '],['']] or len(sentence)==0) ])
    
    #DF_help=DF_help.apply(func=lambda paragraph :[ sentence[1:] if sentence[0]=='' else sentence for sentence in paragraph])
    return DF_help

def remplacer_site(String):       
    for index in [m.start() for m in re.finditer('.', String)]:
        if String[:index].rfind('http')>String[:index].rfind(' '):
            String=String[:index]+'&'+String[index+1:]
    return String

def remplacer_site_DF(DF):
    DF=DF.apply(func= lambda S : remplacer_site(S))
    return DF       

def espacer_point_DF(DF):
    DF=DF.apply(func = lambda S:S.replace('.',' .'))
    return DF

### Création d'un dictionnaire Word2Vec, et encodage du feedbacks sous forme de de vecteurs

In [4]:
def generate_dict(sentences,window,size):
    model=Word2Vec(sentences, size=size, window=window, min_count=1)
    #,negative=1,hs=1
    return model
  
def w2v(DF, dico, size_embeding, size_sentence):
    DF=DF.apply(func = lambda String : np.array([dico[word] for word in String.split(' ')]+[np.zeros(size_embeding) for i in range (size_sentence-len(String.split(' ')))]))
    return DF
"""
cette fonction encode les paragraphe de la colonne feedback sous la forme d'une suite de vecteurs encodant chacun une phrase,
l'encodage d'une phrase est obtenue grâce à la moyenne des encodages des motes composant la phrase. L'encodage pourrait être 
ameilloré en prenant en compte la probabilité qu'un mot soit inclu dans un phrase, plus cette dernière serait faible plus la
pondérantion du mot serait élevée.
"""
def sentence_2v_bis(DF,dico,size_embeding, size_paragraph):
    DF_bis=pd.Series([None for i in range (DF.shape[0])])
    DF_bis.index=DF.index
    for i in range (DF.shape[0]):
        DF_bis.iloc[i]=np.stack([np.average([dico.get_vector(word) for word in sentence], axis=0) for sentence in  DF.iloc[i]]+[np.zeros(size_embeding) for i in range (size_paragraph-len(DF.iloc[i]))],axis=0)
    return DF_bis

### Construit un modèle d'analyse de sentiments

In [5]:
def build_model( size_encoding, max_legnth_para, numClasses, lstmUnits):

    
    model=Sequential()
    model.add(LSTM(120,input_shape=(max_legnth_para,size_encoding), activation='relu', return_sequences=True ))
    model.add(Dropout(0.2))
    model.add(LSTM(120,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(50,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(numClasses,activation='sigmoid'))
    
    opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

    model.compile(
                loss='mean_squared_error',
                optimizer=opt,
                metrics=['mean_absolute_error'],)
    model=model
    return model

### Modèle encodant les artistes sous forme de vecteurs et les influenceurs sous forme de matrice, ce modèle peut être entrainé grâce à la base de donnée fournie

In [16]:
    
class feature_extractor ():
    
    def __init__(self, Data,N_features):
        
        output_data=Data.drop(['influencer_id','acceptation_rate','band_id','date_created']+Type_influencer, axis=1)
        number_output=output_data.shape[1]
        
        influencer_info=Data[['influencer_id','acceptation_rate']+Type_influencer]
        influencer_info=influencer_info.groupby('influencer_id').tail(1)
        
        
        
        band_ids=pd.DataFrame({'band_id':pd.unique(Data['band_id']).tolist()})
        influencer_ids=pd.DataFrame({'influencer_id':pd.unique(Data['influencer_id']).tolist()})
        
        Number_influencer=influencer_ids.shape[0]
        Number_band=band_ids.shape[0]
        
        band_ids['index_of_id']=pd.Series([i for i in range(Number_band)])
        band_ids=band_ids.set_index('band_id')
        influencer_ids['index_of_id']=pd.Series([i for i in range(Number_influencer)])
        influencer_ids=influencer_ids.set_index('influencer_id')
        
        couple=Data[['band_id','influencer_id']]
        couple_index=pd.DataFrame(index=couple.index , columns= ['band_index','influencer_index'])
        couple_index['band_index']=couple['band_id'].apply(func = lambda R : band_ids.loc[R])
        couple_index['influencer_index']=couple['influencer_id'].apply(func = lambda R : influencer_ids.loc[R])
        
        
        band_features=tf.Variable(tf.random.normal([Number_band, N_features+15]))
        
        influencer_info_T=tf.constant(np.repeat(influencer_info.values[np.newaxis,...], [number_output], axis=0), dtype=float)
        influencer_features_help=tf.Variable(tf.random.normal([number_output,Number_influencer,N_features]))
        influencer_features=tf.concat([influencer_info_T,influencer_features_help], axis=2)
        influencer_features=tf.transpose(influencer_features,[0,2,1])
        
        print(influencer_features.shape)
        potential_pred=tf.linalg.matmul(band_features, influencer_features,)
        potential_pred=tf.transpose(potential_pred,perm=[1,2,0])
        pred_vec=tf.gather_nd(potential_pred, list(zip(couple_index['band_index'].tolist(),couple_index['influencer_index'].tolist())))
        
        output_T=tf.constant(output_data.astype(float).values)
        
        loss=mean_squared_error( output_T, pred_vec)
        
        
        
        
        
        self.influencer_features=influencer_features_help
        self.band_features=band_features
        self.loss=loss
        self.band_features=band_features
        self.influencer_features=influencer_features
        self.band_ids=band_ids
        self.influencer_ids=influencer_ids
    
    def optimiser(self,epochs):
        sess = tf.compat.v1.Session()
        opt=tf.train.AdamOptimizer(1e-3).minimize(self.loss)
        sess.run(tf.global_variables_initializer())
        for i in range (epochs) :
            sess.run(opt)
            
        sess.close()
        
    def get_features(self):
        sess = tf.compat.v1.Session()
        sess.run(tf.global_variables_initializer())
        influ_feat=sess.run(self.influencer_features)
        band_feat=sess.run(self.influencer_features)
        influ_feat=[influ_feat[i,:,:]for i in range(influ_feat.shape[0])]
        band_feat=[band_feat[i,:]for i in range(band_feat.shape[0])]
        influ_feat=pd.DataFrame(index=self.influencer_ids.values,data={'feat':influ_feat})
        band_feat=pd.DataFrame(index=self.band_ids.values, data={'feat':band_feat,'id_s':self.band_ids})
        
        band_feat.
        return influ_feat,band_feat
        

        

### prétraitement de la base de données

In [7]:
DF=pd.read_csv('groover_dataset_challenge.csv', index_col = 0)
    
    
DF['feedback']=DF['feedback'].fillna(value='')


DF['decision']=DF['decision'].apply(func = lambda R : np.array(re.sub('[\]\[\']', '', R).split(',')))


#!!!! coprendre ce qu'on fait la
DF=DF.drop('decision', axis = 1).join(DF.decision.str.join('|').str.get_dummies())
DF=DF.drop('influencer_kind', axis = 1).join(pd.get_dummies(DF['influencer_kind']))

DF['acceptation_rate']=DF['acceptation_rate'].fillna(0)

DF['feedback']=DF['feedback'].apply(func = lambda R : R.replace('\n',''))

DF.to_excel("processed_DB.xlsx")

C:\Users\logan\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:931: UserWarning: Ignoring URL 'https://groover.co/media/components/sendButton/sendIcon.pngDu%20bon%20ensemble%20musical%20que%20n'aurait%20pas%20renié%20Santana%20notamment%20sur%20le%20titre%20%22distance%22%20(super%20guitare).%20Je%20trouve%20cependant%20l'humeur%20des%20morceaux%20un%20peu%20légère,%20cela%20repose%20la%20question%20de%20la%20destination%20d'un%20tel%20style%20musical.%20A%20savoir%20ambiancer%20une%20soirée%20dans%20un%20lieu%20commun,%20restaurant%20au%20bord%20du%20lac,%20réception%20?%20Ou%20alors%20pouvoir%20l'écouter%20chez-soi%20de%20manière%20plus%20contemplative%20?%20Ainsi%20la%20résonance%20de%20votre%20musique%20ne%20m'amène%20pas%20plus%20loin%20qu'une%20soirée%20dans%20une%20marina,%20je%20ne%20suis%20pas%20parvenu%20à%20m'extraire%20de%20cette%20dimension%20fonctionnaliste%20que%20votre%20musique%20stimule...%20J'espère%20que%20ce%20retour%20vous%20sera%20utile%20d'une%20manière%20ou

### chargement de la base de donnée

In [8]:
DF=pd.read_excel('processed_DB.xlsx', index_col = 0)
DF['feedback']=DF['feedback'].astype(str)

### création d'un dictionnaire word ->vec et d'un vecteur contenant les encodages des feedback sous forme de vecteurs

In [9]:
window_size=2
embeding_size=100
num_Lstm=120

DF_sentiment=DF.drop(['track__id','band_id','influencer_id','date_created','acceptation_rate']+Type_influencer, axis=1)
feedbacks=DF_sentiment['feedback']

feedbacks=prepare_DF(feedbacks)
feedbacks=prepare_sentences_help(feedbacks)
List=prepare_sentences_List(feedbacks)

num_class=1
Max_size_paragraph=max(feedbacks.apply(func = lambda L : len(L)))

dico=generate_dict(List, window_size, embeding_size)
Dico=dico.wv

path = get_tmpfile("wordvectors.kv")
Dico.save(path)

"""
On encoder les paragraphe de la colonne feedback sous la forme d'une suite de vecteurs encodant chacun une phrase,
l'encodage d'une phrase est obtenue grâce à la moyenne des encodages des motes composant la phrase. L'encodage pourrait être 
ameilloré en prenant en compte la probabilité qu'un mot soit inclu dans un phrase, plus cette dernière serait faible plus la
pondérantion du mot serait élevée.
"""
feedbacks_vec=sentence_2v_bis(feedbacks,Dico,embeding_size,Max_size_paragraph)

### création et entrainement d'un modèle d'analyse de sentiment

In [10]:
Data_sentiment=DF_sentiment
Data_sentiment['feedback']=feedbacks_vec

Xtrain=np.stack(Data_sentiment['feedback'].tolist(),axis=0)
Ytrain=np.stack(Data_sentiment['score'].tolist(),axis=0)
Model=build_model(embeding_size, Max_size_paragraph, num_class, num_Lstm)
Model.fit(Xtrain,Ytrain , epochs=3)

"""
Ce morceau de code permet de mesurer l'éfficacité du modèle avec un set de test, on pourrait utiliser de la cross validation
avoir une vision plus globale de l'efficacité du modèle

Data_Train=Data.iloc[:50000]
Data_Test=Data.iloc[50000:]
Xtrain=np.stack(Data_Train['feedback'].tolist(),axis=0)
Ytrain=np.stack(Data_Train['score'].tolist(),axis=0)
Xtest=np.stack(Data_Test['feedback'].tolist(),axis=0)
Ytest=np.stack(Data_Test['score'].tolist(),axis=0)
Model=build_model(embeding_size, Max_size_paragraph, num_class, num_Lstm)
Model.fit(Xtrain,Ytrain , epochs=3, validation_data=(Xtest,Ytest))
"""

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/3
70346/70346 [==============================] - 127s 2ms/sample - loss: 0.2770 - mean_absolute_error: 0.3527
Epoch 2/3
70346/70346 [==============================] - 130s 2ms/sample - loss: 0.6761 - mean_absolute_error: 0.7065s - loss: 0.6762 - 
Epoch 3/3
70346/70346 [==============================] - 127s 2ms/sample - loss: 0.6751 - mean_absolute_error: 0.7055


"\nCe morceau de code permet de mesurer l'éfficacité du modèle avec un set de test, on pourrait utiliser de la cross validation\navoir une vision plus globale de l'efficacité du modèle\n\nData_Train=Data.iloc[:50000]\nData_Test=Data.iloc[50000:]\nXtrain=np.stack(Data_Train['feedback'].tolist(),axis=0)\nYtrain=np.stack(Data_Train['score'].tolist(),axis=0)\nXtest=np.stack(Data_Test['feedback'].tolist(),axis=0)\nYtest=np.stack(Data_Test['score'].tolist(),axis=0)\nModel=build_model(embeding_size, Max_size_paragraph, num_class, num_Lstm)\nModel.fit(Xtrain,Ytrain , epochs=3, validation_data=(Xtest,Ytest))\n"

### Encodage du feedback grâce aux résultats du modèle d'analyse de sentiments et entrainement des vecteurs représentant les artistes et les influenceurs

In [18]:
Data_matching=DF.drop('date_created', axis=1)
Data_matching['feedback']=feedbacks_vec
Data_matching['feedback']=Data_matching['feedback'].apply(func= lambda feedback : Model.predict(feedback[np.newaxis,...]))

#DF_copy['feedback']=feedbacks_vec.apply(func = lambda R:Model.predict(R))

fe=feature_extractor(Data_matching,10)
fe.optimiser(1)


(31, 25, 554)


## Exercice2



In [19]:
influ_feat,band_feat=fe.get_features()
feat_band_6593=band_feat.loc[6593]
Distance_6593=band_feat.apply(func = lambda F:numpy.linalg.norm(F-feat_band_6593))
10_smallest_dit=Distance_6593.nsmallest(10,'feat')